In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [2]:
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, TimestampNTZType, DecimalType
from binance import Client
import os
from dotenv import load_dotenv, dotenv_values
import logging
import sys
import numpy
sys.path.insert(0,'./pipelines/shared')
from utils import (
    parquet_to_df,
    data_cleaning,
    add_crypto_id,
    add_time_id,
    upload_time,
    upload_price,
)
import pyspark.pandas as ps
import matplotlib as plt
load_dotenv(dotenv_path="./pipelines/shared/main.env", override=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/22 13:38:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/hamza/.local/share/virtualenvs/DE-Crypto-aeKJmBel/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


True

4:80: E501 line too long (84 > 79 characters)
11:18: E231 missing whitespace after ','
12:1: E402 module level import not at top of file
20:1: E402 module level import not at top of file
21:1: E402 module level import not at top of file


In [3]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [4]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .config("spark.jars", "/Users/hamza/Desktop/projects/postgresql-42.7.5.jar") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

25/05/22 13:38:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
3:80: E501 line too long (82 > 79 characters)
7:67: E261 at least two spaces before inline comment
7:67: E262 inline comment should start with '# '
7:80: E501 line too long (101 > 79 characters)
7:102: W291 trailing whitespace
13:80: E501 line too long (80 > 79 characters)


In [5]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #set to debug to capture all levels
if logger.hasHandlers():
    logger.handlers.clear()
logger.propagate = False

2:31: E261 at least two spaces before inline comment
2:32: E262 inline comment should start with '# '


In [6]:
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [7]:
client_minio = Minio(
    "localhost:9000",  # Make sure you're using port 9000 for the S3 API
    # minio_url,
    access_key=MINIO_USER,
    secret_key=MINIO_PASSWORD,
    secure=False,  # Disable SSL if you're not using SSL certificates
)

schema = StructType(
    [
        StructField(name="datetime", dataType=TimestampNTZType(), nullable=False),
        StructField(name="Open Price", dataType=DecimalType(), nullable=False),
        StructField(name="Close Price", dataType=DecimalType(), nullable=False),
        StructField(name="Volume", dataType=DecimalType(), nullable=False),
    ]
)
# doesnt work for parquet files, schema inferred from that instead


read_sql = "SELECT * FROM crypto"
#localhost for local development and container ip for containers
df_crypto = (
    spark.read.format("jdbc")
    .option("url", "jdbc:postgresql://localhost:5432/crypto")
    .option("user", "postgres")
    .option("password", "postgres")
    .option("query", read_sql)
    .option("driver", "org.postgresql.Driver")
    .load()
)

11:80: E501 line too long (82 > 79 characters)
13:80: E501 line too long (80 > 79 characters)
21:1: E265 block comment should start with '# '


In [8]:
df = parquet_to_df(client_minio=client_minio, timeframe="monthly", crypto="BTCUSDT", schema=schema)
df_cleaned = data_cleaning(df)

25/05/22 13:38:03 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/22 13:38:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
1:80: E501 line too long (99 > 79 characters)


In [9]:
#standard deviation - 1 year volatility, all time volatility and rolling volatility
#moving averages
#percentage changes in price and volume
#drawdown
#correlation
#need market index data for beta
#volume rate of change?

1:1: E265 block comment should start with '# '
1:80: E501 line too long (83 > 79 characters)
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:1: E265 block comment should start with '# '
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '
7:1: E265 block comment should start with '# '


In [10]:
ps.set_option("compute.ops_on_diff_frames", True)

In [11]:
psdf = df_cleaned.pandas_api()
psdf_sorted = psdf.sort_values(by='datetime').reset_index(drop = True)


2:63: E251 unexpected spaces around keyword / parameter equals
2:65: E251 unexpected spaces around keyword / parameter equals
3:1: W391 blank line at end of file


In [12]:
psdf_sorted['ema'] = psdf_sorted['close'].ewm(min_periods = 12,ignore_na = True,span = 6).mean()
psdf_sorted = psdf_sorted.sort_values(by='datetime').reset_index(drop = True)
monthly_pct_change = psdf_sorted['close'].pct_change()
monthly_pct_change = monthly_pct_change.round(3)
psdf_sorted['monthly_pct_change'] = monthly_pct_change
std = psdf_sorted['monthly_pct_change'].rolling(window=12).std()
psdf_sorted['rolling_std_12_months'] = std
sma = psdf_sorted['close'].rolling(window=12).mean()
psdf_sorted['sma'] = sma
vroc = (psdf_sorted['volume'] - psdf_sorted['volume'].shift(6))/ psdf_sorted['volume'].shift(6)
psdf_sorted['vroc'] = vroc

1:58: E251 unexpected spaces around keyword / parameter equals
1:60: E251 unexpected spaces around keyword / parameter equals
1:63: E231 missing whitespace after ','
1:73: E251 unexpected spaces around keyword / parameter equals
1:75: E251 unexpected spaces around keyword / parameter equals
1:80: E231 missing whitespace after ','
1:80: E501 line too long (96 > 79 characters)
1:85: E251 unexpected spaces around keyword / parameter equals
1:87: E251 unexpected spaces around keyword / parameter equals
2:70: E251 unexpected spaces around keyword / parameter equals
2:72: E251 unexpected spaces around keyword / parameter equals
10:64: E225 missing whitespace around operator
10:80: E501 line too long (95 > 79 characters)


In [13]:
drawdown = (psdf_sorted['close'].rolling(window=6,min_periods = 6).min() - psdf_sorted['close'].rolling(window=6,min_periods = 6).max())/ psdf_sorted['close'].rolling(window=6).max()

1:50: E231 missing whitespace after ','
1:62: E251 unexpected spaces around keyword / parameter equals
1:64: E251 unexpected spaces around keyword / parameter equals
1:80: E501 line too long (182 > 79 characters)
1:113: E231 missing whitespace after ','
1:125: E251 unexpected spaces around keyword / parameter equals
1:127: E251 unexpected spaces around keyword / parameter equals
1:137: E225 missing whitespace around operator


In [14]:
psdf_sorted['drawdown'] = drawdown

In [ ]:
psdf_sorted

25/05/22 13:38:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/22 13:38:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/22 13:38:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/22 13:38:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/22 13:38:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/22 13:38:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/22 1

Py4JJavaError: An error occurred while calling o1953.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.AbstractStringBuilder.<init>(AbstractStringBuilder.java:101)
	at java.base/java.lang.StringBuilder.<init>(StringBuilder.java:119)
	at org.apache.spark.sql.catalyst.util.StringConcat.toString(StringUtils.scala:62)
	at org.apache.spark.sql.catalyst.util.StringUtils$PlanStringConcat.toString(StringUtils.scala:152)
	at org.apache.spark.sql.execution.QueryExecution.explainString(QueryExecution.scala:254)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.onUpdatePlan(AdaptiveSparkPlanExec.scala:780)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.$anonfun$getFinalPhysicalPlan$2(AdaptiveSparkPlanExec.scala:285)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec$$Lambda/0x0000007001d7c2d8.apply$mcVJ$sp(Unknown Source)
	at scala.runtime.java8.JFunction1$mcVJ$sp.apply(JFunction1$mcVJ$sp.java:23)
	at scala.Opti

In [ ]:
drawdown